In [1]:
import numpy as np
import re
import pandas as pd
import torch
from transformers import AdamW, T5ForConditionalGeneration, T5TokenizerFast as T5Tokenizer, BartForConditionalGeneration, BartTokenizer

2022-06-14 23:54:13.115804: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
data = pd.read_csv('data.csv')

In [3]:
data

,Message,CaseNumber
0,Yes,58340088
1,"No, thank you.",58340088
2,Chat with a person,58340088
3,First Name\nZaira\n\nLast Name\nGarcia\n\nEmai...,58340088
4,Hello! I have the Galaxy Buds - Recently the l...,58340088
...,...,...
418865,Full Name: Jacob Fridley \nContact Number: 276...,53889935
418866,"Sounds good, thank you.",53889935
418867,"Okay, that sounds good. I'll keep an eye out f...",53889935
418868,That should cover it. I appreciate your help.,53889935


In [4]:
dt = data['Message'].copy()
for j in range(len(dt)):
    i = dt.iloc[j]
    if type(i) == type(1) or type(i) == type(1.0):
        dt.iloc[j] = str(i)
data['Message'] = dt

In [5]:
d = data.groupby('CaseNumber')['Message'].apply(lambda x: ' '.join(x))

In [6]:
df = pd.DataFrame(d.values, columns=['Message'])

In [7]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
df['Message'] = df['Message'].apply(deEmojify)

In [8]:
def data_cleaning(msg):
    msg = re.sub('((www\.[^\s]+) | (https?://[^\s]+))', '', msg) # removing links
    msg = re.sub('([^\s]*@[^\s]+)', '', msg)         # removing usernames, emails
    msg = re.sub('([\s][+]?[0-9]+)', ' ', msg) # removing numbers
    return msg

In [9]:
df['Message'] = df['Message'].apply(data_cleaning)

In [10]:
for i in df.index:
    if len(df.loc[i, 'Message']) > 1000:
        df.drop(i, inplace = True)

In [11]:
df = df[:1000]

In [12]:
df

,Message
0,HELP Mobile device Need shipping label Okay I...
1,I need a refrigerator repair service call I ne...
4,My glass stovetop cracked while cooking Okay n...
6,Is the Galaxy S2 watch compatible with the iPh...
9,Our Samsung refrigerator lower ice maker is br...
...,...
1350,First Name\nMiguel\n\nLast Name\nBermudez\n\nE...
1351,Support \n Random white pixels across the scr...
1352,SUPPORT Battery doesn't hold charge Hello my...
1353,I am not receiving or able to send out any ki...


### Predicting Summaries using Fine Tuned T5 - 1

In [13]:
summary = pd.DataFrame(df['Message'], columns=['Message'])

In [14]:
def summarizeText(text):
    text_encoding = tokenizer(
        text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = model.generate(
        input_ids=text_encoding['input_ids'],
        attention_mask=text_encoding['attention_mask'],
        max_length=150,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)

In [15]:
pretrained_model = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(pretrained_model)
finetuned_model = 'vaibhavagg303/T5-test'
model = T5ForConditionalGeneration.from_pretrained(finetuned_model, return_dict=True)

/home/vaibhav/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [16]:
summary['Summary_T5_1'] = summary['Message'].apply(summarizeText)

In [25]:
pretrained_model = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(pretrained_model)
finetuned_model = 'vaibhavagg303/T5-test2'
model = T5ForConditionalGeneration.from_pretrained(finetuned_model, return_dict=True)

/home/vaibhav/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [26]:
i = 1
def summarizeText(text):
    global i 
    print(i)
    i += 1
    text_encoding = tokenizer(
        text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = model.generate(
        input_ids=text_encoding['input_ids'],
        attention_mask=text_encoding['attention_mask'],
        max_length=150,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)

In [27]:
summary['Summary_T5_2'] = summary['Message'].apply(summarizeText)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [19]:
finetuned_model = 'vaibhavagg303/Bart-Fine-Tuned2'
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained(finetuned_model)

In [20]:
i = 1
def summarizeText(text):
    global i 
    print(i)
    i += 1
    text_encoding = tokenizer(
        text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = model.generate(
        input_ids=text_encoding['input_ids'],
        attention_mask=text_encoding['attention_mask'],
        max_length=150,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)

In [21]:
summary['Summary_Bart'] = summary['Message'].apply(summarizeText)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [28]:
summary.to_csv('summary1.csv', index=None)

In [30]:
test_row = summary.iloc[0]
msg = test_row['Message']
s3 = test_row['Summary_Bart']
s2 = test_row['Summary_T5_2']
s1 = test_row['Summary_T5_1']

In [31]:
s1

'The UPS sent back my phone saying I had the wrong shipping label. So I am needing the correct one sent to me Yes Okay I got the email. Thank you so much!'

In [32]:
s2

'HELP mobile phone need shipping label'

In [33]:
s3

'HELP Mobile device  Need shipping label Okay I am suppose to be shipping back my phone to be worked on and the UPS sent it back saying I had the wrong shipping label. So I am needing the correct one sent to me Yes  Okay I got the email. Thank you so much! Okay, thank you! Got it No thank u Same to you   Yes First time'

In [34]:
msg

'HELP Mobile device  Need shipping label Okay I am suppose to be shipping back my phone to be worked on and the UPS sent it back saying I had the wrong shipping label. So I am needing the correct one sent to me Yes  Okay I got the email. Thank you so much! Okay, thank you! Okay, got it No thank u Same to you   Yes First time'